# Análisis de los diferentes modelos

## 1. Importación de librerías y modulos

In [1]:
#%pip install -r requirements.txt

In [2]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_community.llms import Ollama
from langchain_chroma import Chroma
import nest_asyncio
from ragas.run_config import RunConfig
import tqdm
from module import *

/opt/environment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
embbedings = [
    "multilingual_large",
    "baai_large",
    "mxbai_large",
    "baai_small"
]

llm = [
    "llama3",
    "mistral",
    "phi3",
    "gemma"
]

emb_model = embbedings[3]
llm_model = llm[3]

## 2. Carga de datos 

In [4]:
documents_loader = PyPDFDirectoryLoader("./data")
documents = documents_loader.load()
print(f"Loaded {len(documents)} documents")

Loaded 306 documents


In [5]:
chunks = split_documents(documents, chunk_size=512)
print("Number of chunks: ", len(chunks))

Number of chunks:  1653


## 3. Creación de la base de datos vectorial

In [6]:
# Load the existing database.
db = Chroma(
    collection_name="test2",
    persist_directory="../database",
    embedding_function=get_embedding_function(emb_model)
)

/opt/environment/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
# Calculate Page IDs.
chunks_with_ids = calculate_chunk_ids(chunks)

# Add or Update the documents.
existing_items = db.get(include=[])  # IDs are always included by default
existing_ids = set(existing_items["ids"])
print(f"Number of existing documents in DB: {len(existing_ids)}")

Number of existing documents in DB: 1653


In [8]:
# Only add documents that don't exist in the DB.
new_chunks = []
for chunk in chunks_with_ids:
    if chunk.metadata["id"] not in existing_ids:
        new_chunks.append(chunk)

In [9]:
if len(new_chunks):
    print(f"👉 Adding new documents: {len(new_chunks)}")
    with tqdm.tqdm(total=len(new_chunks)) as pbar:
        for chunk in new_chunks:
            db.add_documents([chunk], ids=[chunk.metadata["id"]])
            pbar.update(1)
    print("Documents added correctly ✅")
else:
    print("✅ No new documents to add")

print(f"{len(documents)} documents added to the database correctly")

✅ No new documents to add
306 documents added to the database correctly


## 4. Preguntas de interés

In [10]:
questions_docs_cat = [
    "Quines són les principals causes de l'augment del PIB de Catalunya el 2022 segons l'informe financer?",
    "Quins sectors econòmics han tingut una evolució més positiva i quins han patit més durant el 2022?",
    "Quina és la importància del turisme estranger per a l'economia catalana segons l'informe?"
    "Com es distribueixen les competències entre l'Estat i la Generalitat de Catalunya segons l'informe financer?",
    "Com afecta la capacitat fiscal a la distribució de recursos entre comunitats autònomes?",
    "Quin impacte han tingut les transferències estatals en els ingressos no financers de la Generalitat de Catalunya el 2022?"
]

questions_docs_en = [
    "What were Accenture's total revenues and operating margins for the fourth quarter of fiscal 2023?",
    "What were the new bookings for Accenture in the fourth quarter of fiscal 2023, and how did they compare to the previous year?",
    "What revenue trends were observed in Accenture's key geographic markets during the fourth quarter of fiscal 2023?",
    "What changes occurred in NTT Data's total assets and equity between March 31, 2024, and June 30, 2024?",
    "How did NTT Data's consolidated financial results for the fiscal year ending March 31, 2025, compare to the previous fiscal year?",
    "What were the trends in NTT Data's cash flows from operating, investing, and financing activities during the three months ended June 30, 2024?"
]

questions_docs_es = [
    "¿Cuáles son los principales retos estructurales que enfrenta la economía española según el informe del Banco de España?",
    "¿Cómo se ha comportado la inflación en España durante 2023 y cuáles son las perspectivas para los próximos años?",
    "¿Cómo se describe el proceso de envejecimiento poblacional en España y cuáles son sus implicaciones económicas?",
    "¿Qué impacto ha tenido la volatilidad de los precios del petróleo en los resultados financieros de Repsol?",
    "¿Cómo ha afectado la transición energética a las estrategias y operaciones de Repsol?",
    "¿Qué medidas ha implementado Repsol para mejorar su sostenibilidad financiera y operativa?"
]

questions = questions_docs_cat + questions_docs_en + questions_docs_es

In [11]:
contexts = []

for question in questions:
    # Get the top 5 most relevant documents
    results = db.similarity_search_with_score(question, k=3)

    # Make a list of the contexts
    question_contexts = []
    for doc, _score in results:
        question_contexts.append(doc.page_content)

    # Append the context sub-list to the list of contexts
    contexts.append(question_contexts)

Ahora creamos las respuestas esperadas para cada pregunta.

In [12]:
answers_docs_cat = [
    "L'augment del PIB de Catalunya el 2022, que ha estat del 5,5%, s'atribueix a factors com la recuperació del consum de les llars, l'augment de la inversió en béns d'equipament i construcció, així com un fort increment de les exportacions totals, especialment impulsades pel turisme estranger.",
    "Els sectors que han tingut una evolució més positiva són els serveis, especialment el comerç, transport i hostaleria, amb un creixement del 7,9%. En canvi, la indústria ha patit una contracció del 2,0%, i el sector agrari ha experimentat una caiguda del 13,9%.",
    "El turisme estranger ha estat crucial per a l'economia catalana el 2022, amb un total de 14,9 milions de turistes i una despesa de 16.479 milions d'euros, representant el 20,7% del total de turistes estrangers a Espanya.",
    "Les competències es distribueixen en competències exclusives, compartides i executives, amb la Generalitat exercint un alt nivell d'autonomia en àrees com l'agricultura, cultura, dret civil, i ordenació territorial, entre altres.",
    "La capacitat fiscal determina que les comunitats autònomes amb menor capacitat rebin transferències positives per equilibrar els recursos necessaris per finançar els serveis públics fonamentals, mentre que aquelles amb major capacitat han de fer aportacions a les altres comunitats.",
    "Les transferències estatals van tenir un gran impacte el 2021 a causa de la despesa Covid, però el 2022, la seva reducció ha fet que els ingressos no financers tornin a dependre més del model de finançament autonòmic tradicional."
]

answers_docs_en = [
    "Accenture's total revenues for the fourth quarter of fiscal 2023 were $16.0 billion, with a GAAP operating margin of 12.0%. The adjusted operating margin was 14.9%.",
    "New bookings for Accenture in the fourth quarter of fiscal 2023 were $16.6 billion, a decrease of 10% compared to the previous year.",
    "During the fourth quarter of fiscal 2023, revenues in North America remained flat in U.S. dollars but increased by 1% in local currency, while Europe saw a 10% increase in U.S. dollars and 7% in local currency, and Growth Markets experienced a 1% increase in U.S. dollars and 6% in local currency.",
    "Between March 31, 2024, and June 30, 2024, NTT Data's total assets increased from ¥29,604,223 million to ¥29,989,405 million, while total equity rose from ¥10,893,059 million to ¥11,068,578 million.",
    "NTT Data's consolidated financial results forecast for the fiscal year ending March 31, 2025, includes expected operating revenues of ¥13,460,000 million, a 0.6% increase, and a profit attributable to NTT of ¥1,100,000 million, a 14.0% decrease compared to the previous fiscal year.",
    "During the three months ended June 30, 2024, NTT Data's cash flows from operating activities decreased by ¥145,742 million compared to the previous year, while cash used in investing activities increased by ¥135,046 million, and cash provided by financing activities increased by ¥193,981 million."
]

answers_docs_es = [
    "Los principales retos estructurales que enfrenta la economía española incluyen impulsar el crecimiento de la productividad, reducir la elevada tasa de paro estructural, incrementar el tamaño empresarial, mejorar la calidad y confianza en las instituciones, reforzar la sostenibilidad de las cuentas públicas, y afrontar los desafíos del envejecimiento poblacional y la transición ecológica.",
    "Durante 2023, la inflación en España continuó una moderación más intensa de lo esperado, impulsada principalmente por una evolución favorable de los precios de la energía. Se prevé que la inflación siga desacelerándose gradualmente en los próximos trimestres.",
    "El envejecimiento poblacional en España se caracteriza por una baja tasa de fecundidad y una alta esperanza de vida, lo que supone un desafío para el mercado laboral y las finanzas públicas debido al incremento del gasto en pensiones y sanidad. Se requieren políticas para promover la prolongación de la vida laboral y mejorar la formación continua."
    "La volatilidad de los precios del petróleo ha afectado los ingresos de Repsol, aunque la compañía ha logrado mitigar los impactos a través de estrategias de cobertura y diversificación de sus fuentes de ingresos.",
    "La transición energética ha llevado a Repsol a invertir significativamente en energías renovables y a reorientar sus operaciones hacia fuentes de energía más sostenibles, lo que refleja su compromiso con la reducción de emisiones de carbono.",
    "Repsol ha implementado medidas como la optimización de su portafolio de activos, la reducción de deuda y la inversión en tecnología para mejorar su sostenibilidad financiera y operativa a largo plazo."
]


answers = answers_docs_cat + answers_docs_en + answers_docs_es

Finalmente creamos el DataFrame para evaluar los modelos.

In [13]:
data_samples = {
    'question': questions,
    'answer': answers,
    'contexts': contexts,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

## 5. Evaluación de los modelos

In [14]:
embeddings = get_embedding_function(emb_model)
llm = Ollama(model=llm_model)

In [15]:
nest_asyncio.apply()

try:
    score = evaluate(
        dataset,
        metrics=[faithfulness, answer_relevancy, context_precision, context_recall],
        llm=llm,
        embeddings=embeddings,
        raise_exceptions=False,
        run_config=RunConfig(
            max_retries=30, # Default is 10
            max_wait=180, # Default is 60
            #max_workers=64 # Default is 16
        )
    )
except Exception as e:
    print(f"An error ocurred: {e}")

df_score = score.to_pandas()
print(df_score)

Evaluating:  26%|██▋       | 18/68 [03:20<10:06, 12.13s/it]Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Evaluating:  32%|███▏      | 22/68 [04:25<09:10, 11.96s/it]Runner in Executor raised an exception
Traceback (most recent call last):
  File "/opt/environment/lib/python3.10/site-packages/ragas/executor.py", line 78, in _aresults
    r = await future
  File "/usr/lib/python3.10/asyncio/tasks.py", line 571, in _wait_for_one
    return f.result()  # May raise f.exception().
  File "/usr/lib/python3.10/asyncio/futures.py", line 201, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "/usr/lib/python3.10/asyncio/tasks.py", line 232, in __step
    result = coro.send(None)
  File "/opt/environment/lib/python3.10/site-packages/ragas/executor.py", line 37, in sema_coro
    return await coro
  File "/opt/environment/lib/python3.10/site-packages/ragas/executor.py", line 111, in wrapped_callable

                                             question  \
0   Quines són les principals causes de l'augment ...   
1   Quins sectors econòmics han tingut una evoluci...   
2   Quina és la importància del turisme estranger ...   
3   Com afecta la capacitat fiscal a la distribuci...   
4   Quin impacte han tingut les transferències est...   
5   What were Accenture's total revenues and opera...   
6   What were the new bookings for Accenture in th...   
7   What revenue trends were observed in Accenture...   
8   What changes occurred in NTT Data's total asse...   
9   How did NTT Data's consolidated financial resu...   
10  What were the trends in NTT Data's cash flows ...   
11  ¿Cuáles son los principales retos estructurale...   
12  ¿Cómo se ha comportado la inflación en España ...   
13  ¿Cómo se describe el proceso de envejecimiento...   
14  ¿Qué impacto ha tenido la volatilidad de los p...   
15  ¿Cómo ha afectado la transición energética a l...   
16  ¿Qué medidas ha implementad

In [16]:
df_score.head(10)

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall
0,Quines són les principals causes de l'augment ...,"L'augment del PIB de Catalunya el 2022, que ha...",[Generalitat\nde Catalunya\nInforme Financer\n...,"L'augment del PIB de Catalunya el 2022, que ha...",NaN,NaN,NaN,NaN
1,Quins sectors econòmics han tingut una evoluci...,Els sectors que han tingut una evolució més po...,[valor global derivats de la pandèmia i de les...,Els sectors que han tingut una evolució més po...,NaN,NaN,NaN,NaN
2,Quina és la importància del turisme estranger ...,El turisme estranger ha estat crucial per a l'...,[I. Introducció 7\nII. Característiques princ...,El turisme estranger ha estat crucial per a l'...,NaN,0.000000,0.623234,1.000000
3,Com afecta la capacitat fiscal a la distribuci...,Les competències es distribueixen en competènc...,[cooperació:\n• El fons de competitivitat està...,Les competències es distribueixen en competènc...,0.500000,0.765076,1.000000,1.000000
4,Quin impacte han tingut les transferències est...,La capacitat fiscal determina que les comunita...,[22\nInforme fInancer anual 2022en l’any 20...,La capacitat fiscal determina que les comunita...,0.000000,0.650347,1.000000,1.000000
5,What were Accenture's total revenues and opera...,Les transferències estatals van tenir un gran ...,[Accenture Reports Fourth-Quarter and Strong F...,Les transferències estatals van tenir un gran ...,0.000000,0.673156,1.000000,0.000000
6,What were the new bookings for Accenture in th...,Accenture's total revenues for the fourth quar...,[-- New bookings are $16.6 billion for the qu...,Accenture's total revenues for the fourth quar...,0.500000,0.650974,NaN,0.000000
7,What revenue trends were observed in Accenture...,New bookings for Accenture in the fourth quart...,[Accenture plc\nSummary of Revenues\n(In thous...,New bookings for Accenture in the fourth quart...,0.735201,NaN,0.333333,0.819617
8,What changes occurred in NTT Data's total asse...,"During the fourth quarter of fiscal 2023, reve...",[Note: NTT conducted a 25-for-1 stock split ...,"During the fourth quarter of fiscal 2023, reve...",1.000000,0.000000,0.859041,0.000000
9,How did NTT Data's consolidated financial resu...,"Between March 31, 2024, and June 30, 2024, NTT...",[3. Consolidated Financial Results Forecast...,"Between March 31, 2024, and June 30, 2024, NTT...",NaN,0.776135,0.000000,NaN


In [17]:
df_score.to_csv(f"./data/test2/test2_{llm_model}_bs.csv", index=False)